あるWEBサイトは、新しいウェブページを設計し、それが従来のウェブページよりパフォーマンスが向上されたかどうかを検証することを望んでいる。テストは既に完了しており、次に収集されたデータを使用してABテストで結果を検証する。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import norm

### １.データ導入

In [2]:
df_abdata = pd.read_csv('./ab_data.csv')
df_abdata.head(6)

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
5,936923,2017-01-10 15:20:49.083499,control,old_page,0


In [3]:
df_abdata.shape

(294478, 5)

In [4]:
df_countries = pd.read_csv('./countries.csv')
df_countries.head(6)

,user_id,country
0,834778,UK
1,928468,US
2,822059,UK
3,711597,UK
4,710616,UK
5,909908,UK


In [5]:
df_countries.shape

(290584, 2)

+ ab_data.csv:ユーザーID、タイムスタンプ、グループ、および最終的な効果（ＣＶ）データ
+ countries.csv:各ユーザーの所在国データ

### ２.データクレンジング

In [6]:
df_abdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [7]:
df_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290584 entries, 0 to 290583
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  290584 non-null  int64 
 1   country  290584 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.4+ MB


1.両方ともnull値はない。

2.df_abdataのid数は294478で、df_countriesのid数の290584よりも多い。

可能な原因として：
+ df_abdataに何回も記録されたユーザーがいる。
+ df_abdataの中には、所在国のデータがdf_countriesに存在しないユーザーがいる。

In [8]:
print('abdata:',df_abdata['user_id'].nunique())
print('countrydata:',df_countries['user_id'].nunique())

abdata: 290584
countrydata: 290584


1. ユニークなユーザー数は同じため、ユーザー数は合致していない原因として、df_abdataに何回も記録されたユーザーがいる可能性は高いと考えられる。
   - ただし、abdataのuser_idとcountrydataのuser_idは完全に一致している保証はなく、寧ろ共通部分が一部だけの場合が普通だと思う。


2. countrydataのユーザー数とユニークユーザー数は同じため、重複値はないと考えられる。

In [9]:
print(df_abdata.duplicated().sum())

print(df_abdata.duplicated(subset = 'user_id').sum())

0
3894


In [10]:
t_old_error = df_abdata.query('group == "treatment" and landing_page == "old_page"')

c_new_error = df_abdata.query('group == "control" and landing_page == "new_page"')

print(f'Treatment samples viewed old page:{len(t_old_error)}, Control samples viewed new page:{len(c_new_error)}')

Treatment samples viewed old page:1965, Control samples viewed new page:1928


ところで、まず記録にミスがあるデータを排除する。

このようなデータ（新ウェブページを見るはずなのに、旧ページ見てしまったユーザーと旧ウェブページを見るはずなのに、新ページ見てしまったユーザー）はそれぞれ1038人と1006人である。

In [11]:
error = list(t_old_error.index) + list(c_new_error.index)
df_abdata.drop(index = error, inplace = True)

df_abdata.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [12]:
print(df_abdata.duplicated(subset = 'user_id').sum())

1


エーラデータを削除後、重複データは一個しか残っていない。

In [13]:
df_abdata.drop_duplicates(subset = 'user_id', keep = 'first', inplace = True)

重複したユーザーは最初のデータしか保存しない。

In [14]:
print(df_abdata.duplicated(subset = 'user_id').sum())

0


In [15]:
df_abdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290584 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       290584 non-null  int64 
 1   timestamp     290584 non-null  object
 2   group         290584 non-null  object
 3   landing_page  290584 non-null  object
 4   converted     290584 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 13.3+ MB


In [16]:
df_abdata.shape

(290584, 5)

削除後、残りのデータ数290584。

In [17]:
df_new = pd.merge(df_abdata, df_countries, on='user_id', how='left')
df_new.head()

,user_id,timestamp,group,landing_page,converted,country
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,US
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,US
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,US
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,US
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,US


2つのデータセットをマージする。

In [18]:
df_new.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
country         0
dtype: int64

所在国の記録はnullのデータはない

In [19]:
df_new.groupby(by = 'country').agg({'country':'count'})

,country
country,
CA,14499
UK,72466
US,203619


### CVR計算

In [20]:
def get_cvr(df):
    t_sample = df[df['group'] == 'treatment']
    c_sample = df[df['group'] == 'control']
    t_num = len(t_sample)
    c_num = len(c_sample)
    print(f'Treatment num: {t_num}, Control num: {c_num}')
# treatmentグループとcontrolグループのサンプル数を計算する
    

    t_converted = t_sample.converted == 1
    c_converted = c_sample.converted == 1
    t_converted_num = (t_converted).sum()
    c_converted_num = (c_converted).sum()
    print(f'Converted treatment num: {t_converted_num}, Converted control num: {c_converted_num}')
# treatmentグループとcontrolグループのそれぞれcv数を計算する
    
    return t_sample, c_sample, t_num, c_num, t_converted, c_converted, t_converted_num, c_converted_num

In [21]:
t_sample, c_sample, t_num, c_num, t_converted, c_converted, t_converted_num, c_converted_num = get_cvr(df_new)

t_std = t_converted.std()
c_std = c_converted.std()

# pandasのstdのデフォルト設定は不偏標準偏差。

print(f'Converted treatment std: {t_std:.4f}, Converted control std: {c_std:.4f}')
t_cr = t_converted.mean()
c_cr = c_converted.mean()
print(f'Converted treatment rate: {t_cr:.4f}, Converted control rate: {c_cr:.4f}')

Treatment num: 145310, Control num: 145274
Converted treatment num: 17264, Converted control num: 17489
Converted treatment std: 0.3236, Converted control std: 0.3254
Converted treatment rate: 0.1188, Converted control rate: 0.1204


### 仮説検定

旧ページのCVRをp1、新ページのCVRを2にする。

H0: p1 = p2

H1: p1 < p2

有意水準を5％にする。

In [22]:
z_score1, pvalue1 = proportions_ztest([c_converted_num, t_converted_num],[c_num, t_num], alternative='smaller')
print(f"Z score: {z_score1:.4f}, P value: {pvalue1:.4f}")

alpha = 0.05
print("95% level critial value(one-sided):", norm.ppf(alpha)) 

Z score: 1.3109, P value: 0.9051
95% level critial value(one-sided): -1.6448536269514729


Z score は明らかに-1.6448536269514729から離れているため、帰無仮説を棄却できない。

新ページのパフォーマンスが旧ページより優れているとはいえない。

### Cohen's d

In [23]:
std_pooled = np.sqrt(((t_num - 1) * t_std ** 2 + (c_num - 1) * c_std ** 2) / (t_num + c_num -2))

In [24]:
d = (t_cr - c_cr)/std_pooled

abs(d)

0.004863753093161925

Cohen's dの絶対値は0.2より小さいため、効果はわずかであると考えられる。

### 組分けて再度チェック

サンプル全体的な効果はわずかであるが、所在国を分けて、各国内部の効果を検証すると、異なる結果が出る可能性があるため、所在国ごとに再検証してみる。

In [25]:
def abtest_bycountry(df):
    for i in set(df_new['country']):
        df_country = df[df['country'] == i]
        print(f'A/B Test for {i}')
        t_sample, c_sample, t_num, c_num, t_converted, c_converted, t_converted_num, c_converted_num = get_cvr(df_country)
        t_cr = t_converted.mean()
        c_cr = c_converted.mean()
        diff = t_cr - c_cr
        std_pooled = np.sqrt(((t_num - 1) * t_std ** 2 + (c_num - 1) * c_std ** 2) / (t_num + c_num -2))
        z_score1, pvalue1 = proportions_ztest([c_converted_num, t_converted_num],[c_num, t_num], alternative='smaller')
        d = diff/std_pooled
        print(f'Z score: {z_score1:.4f}, P value: {pvalue1:.4f}' , '\n', 
             f'Cohen\'d : {d:.4f}', '\n')

abtest_bycountry(df_new)

A/B Test for CA
Treatment num: 7301, Control num: 7198
Converted treatment num: 817, Converted control num: 855
Z score: 1.2969, P value: 0.9027 
 Cohen'd : -0.0212 

A/B Test for UK
Treatment num: 36106, Control num: 36360
Converted treatment num: 4375, Converted control num: 4364
Z score: -0.4749, P value: 0.3174 
 Cohen'd : 0.0035 

A/B Test for US
Treatment num: 101903, Control num: 101716
Converted treatment num: 12072, Converted control num: 12270
Z score: 1.5052, P value: 0.9339 
 Cohen'd : -0.0067 



統計的に有意な結果は示されていないので、

新ページのパフォーマンスが旧ページより優れているとは言いにくい。